In [ ]:
%pip install nltk scikit-learn numpy pandas

# Download required NLTK data
import nltk
nltk.download('stopwords')
nltk.download('punkt')

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 10.8 MB/s  0:00:00
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   ------------------- -------------------- 4.2/8.7 MB 20.9 MB/s eta 0:00:01
   -------------------------------------- - 8.4/8.7 MB 20.4 MB/s eta 0:00:01
   ---------------------------------------- 8.7/8.7 MB 19.7 MB/s  0:00:00
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   ------------- -------------------------- 4.2/12.8 MB 20.8 MB/s eta 0:00:01
   -------------------------- ------------- 8.4/12.8 MB 20.1 MB/s eta 0:00:01
   ---------------------------------------  12.6/12.8 MB 20.2 MB/s eta 0:00:01
   ---------------------------------------- 12.8/12.8 MB 19.7 MB/s  0:00:00
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   --------------- ------------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import os
import pickle
import email_read_util

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\Asus/nltk_data'
    - 'c:\\Python313\\nltk_data'
    - 'c:\\Python313\\share\\nltk_data'
    - 'c:\\Python313\\lib\\nltk_data'
    - 'C:\\Users\\Asus\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


## Download 2007 TREC Public Spam Corpus
1. Read the "Agreement for use"
   https://plg.uwaterloo.ca/~gvcormac/treccorpus07/

2. Download 255 MB Corpus (trec07p.tgz) and untar into the 'chapter1/datasets' directory

3. Check that the below paths for 'DATA_DIR' and 'LABELS_FILE' exist

In [ ]:
DATA_DIR = 'datasets/trec07p/data/'
LABELS_FILE = 'datasets/trec07p/full/index'
TRAINING_SET_RATIO = 0.7

In [ ]:
labels = {}
spam_words = set()
ham_words = set()

In [ ]:
# Read the labels
with open(LABELS_FILE) as f:
    for line in f:
        line = line.strip()
        label, key = line.split()
        labels[key.split('/')[-1]] = 1 if label.lower() == 'ham' else 0

In [ ]:
# Split corpus into train and test sets
filelist = os.listdir(DATA_DIR)
X_train = filelist[:int(len(filelist)*TRAINING_SET_RATIO)]
X_test = filelist[int(len(filelist)*TRAINING_SET_RATIO):]

In [ ]:
if not os.path.exists('blacklist.pkl'):
    for filename in X_train:
        path = os.path.join(DATA_DIR, filename)
        if filename in labels:
            label = labels[filename]
            stems = email_read_util.load(path)
            if not stems:
                continue
            if label == 1:
                ham_words.update(stems)
            elif label == 0:
                spam_words.update(stems)
            else:
                continue
    blacklist = spam_words - ham_words
    pickle.dump(blacklist, open('blacklist.pkl', 'wb'))
else:
    blacklist = pickle.load(open('blacklist.pkl', 'rb') )

print('Blacklist of {} tokens successfully built/loaded'.format(len(blacklist)))

Blacklist of 97939 tokens successfully built/loaded


In [ ]:
from nltk.corpus import words
word_set = set(words.words())
word_set.intersection(blacklist)

{'pectora',
 'sleet',
 'soma',
 'sorb',
 'raglan',
 'pluma',
 'thrower',
 'ducal',
 'vatman',
 'biaxial',
 'choral',
 'muzz',
 'merk',
 'degum',
 'lino',
 'punctual',
 'zig',
 'whopper',
 'saunter',
 'commot',
 'pian',
 'enchant',
 'starlit',
 'handmaid',
 'matchbook',
 'chil',
 'prote',
 'cush',
 'feme',
 'cerulean',
 'flamenco',
 'fie',
 'adroit',
 'calor',
 'electrician',
 'batik',
 'depositor',
 'ammonia',
 'dor',
 'humph',
 'throb',
 'osteosarcoma',
 'phylum',
 'staphylococci',
 'redub',
 'romaunt',
 'castrum',
 'billyboy',
 'snug',
 'phosphor',
 'caroli',
 'sweatband',
 'sequent',
 'flutter',
 'abac',
 'subjunct',
 'matriarch',
 'trisect',
 'hazelwood',
 'crept',
 'sienna',
 'camelopard',
 'austral',
 'amaranth',
 'embank',
 'briar',
 'fishtail',
 'shou',
 'chessboard',
 'reman',
 'ponent',
 'coxa',
 'ornithologist',
 'colter',
 'ammono',
 'bajada',
 'heterotopia',
 'cand',
 'rainless',
 'nar',
 'polka',
 'burrow',
 'augend',
 'irrupt',
 'quarrel',
 'nothing',
 'goldfish',
 'fibr

In [ ]:
fp = 0
tp = 0
fn = 0
tn = 0

for filename in X_test:
    path = os.path.join(DATA_DIR, filename)
    if filename in labels:
        label = labels[filename]
        stems = email_read_util.load(path)
        if not stems:
            continue
        stems_set = set(stems)
        if stems_set & blacklist:
            if label == 1:
                fp = fp + 1
            else:
                tp = tp + 1
        else:
            if label == 1:
                tn = tn + 1
            else:
                fn = fn + 1

In [ ]:
from IPython.display import HTML, display
conf_matrix = [[tn, fp],
               [fn, tp]]
display(HTML('<table><tr>{}</tr></table>'.format(
    '</tr><tr>'.join('<td>{}</td>'.format(
        '</td><td>'.join(str(_) for _ in row)) 
                     for row in conf_matrix))))

6772,714
5835,7543


In [ ]:
count = tn + tp + fn + fp
percent_matrix = [["{:.1%}".format(tn/count), "{:.1%}".format(fp/count)],
                  ["{:.1%}".format(fn/count), "{:.1%}".format(tp/count)]]
display(HTML('<table><tr>{}</tr></table>'.format(
    '</tr><tr>'.join('<td>{}</td>'.format(
        '</td><td>'.join(str(_) for _ in row)) 
                     for row in percent_matrix))))

32.5%,3.4%
28.0%,36.2%


In [ ]:
print("Classification accuracy: {}".format("{:.1%}".format((tp+tn)/count)))

Classification accuracy: 68.6%
